## Import Python Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
# !unzip "drive/MyDrive/Skin-detector/Mpox-skin.zip" -d "drive/MyDrive/Skin-detection"

# End-to-end Skin Disease Detection
This model builds by Group3people image dectector using Tensorflow.

## 1.Problem
  Identifying types of skin which is good or bad, and give the percentage of disease a model can learn and give us information.
## 2.Data
  The data I'm using is a part of Kaggle project Mbox Skin Lesion Dataset
  https://www.kaggle.com/datasets/joydippaul/mpox-skin-lesion-dataset-version-20-msld-v20/data
## 3.Evaluation
  The evaluation is a file with prediction probabilities for each skin disease of each test image
## 4.Features
  Some informations about the data:
  * I'm dealing with images (unstructured data) so it's probably best I use deep learning/transfer learning
  * There are 5 diseases of skin which is commonly seen in the real life and 1 for healthy skin
  * There are 5 folders which it uses to train the model but seperately (for only 1 folder will be used at a runtime when training the model)and over 450+ images which i used in this project

### Get workspace ready

* Import TensorFlow 2.0
* Import TensorFlow Hub
* Make sure using the GPU

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
# print(tf.__version__)
# print(hub.__version__)
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model
from PIL import Image

# print("GPU", "available") if tf.config.list_physical_devices("GPU") else print("not available")

### Getting Data Ready
With all machine learning models, data has to be numberical format. So that's all I need to do first, it called numberical representations.

Now start by accessing the data and checking out the labels

In [ ]:
# dataset_path = 'drive/MyDrive/Skin-detection/Original Images/Original Images/FOLDS/fold3'
# selected_class = ['Chickenpox',
#                   'Cowpox',
#                   'HFMD',
#                   'Healthy',
#                   'Measles',
#                   'Monkeypox',]

# output_dir = 'drive/MyDrive/Skin-detection/working/filtered_dataset'
# os.makedirs(output_dir, exist_ok=True)
# for folder in ['Train', 'Test'] :
#   folder_path = os.path.join(dataset_path, folder)
#   output_folder_path = os.path.join(output_dir, folder)
#   os.makedirs(output_folder_path, exist_ok=True)
#   for class_name in selected_class:
#     class_dir = os.path.join(folder_path, class_name)
#     output_class_dir = os.path.join(output_folder_path, class_name)
#     os.makedirs(output_class_dir, exist_ok=True)
#     for filename in os.listdir(class_dir) :
#       src_path = os.path.join(class_dir, filename)
#       dst_path = os.path.join(output_class_dir, filename)
#       shutil.copy(src_path, dst_path)

filtered_train_dir = 'drive/MyDrive/Skin-detection/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train'
filtered_test_dir  = 'drive/MyDrive/Skin-detection/Original Images/Original Images/FOLDS/fold1/Test'

width_size = 100
height_size = 100
color = 3
epoch_number = 10

filtered_train_datagen = ImageDataGenerator(rescale=1. / 255,
                                            rotation_range=40,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode='nearest')
filtered_train_generator = filtered_train_datagen.flow_from_directory(filtered_train_dir,
                                                                      target_size=(width_size, height_size),
                                                                      class_mode='categorical')
filtered_test_datagen = ImageDataGenerator(rescale=1. / 255,
                                           rotation_range=40,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True,
                                           fill_mode='nearest')
filtered_test_generator = filtered_test_datagen.flow_from_directory(filtered_test_dir,
                                                                    target_size=(width_size, height_size),
                                                                    class_mode='categorical')
class_labels = filtered_train_generator.class_indices

# CNN Architecture
# Initialize the model
filtered_model = Sequential()

# Add convolutional layers
filtered_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width_size, height_size, color)))
filtered_model.add(MaxPooling2D((2, 2)))

filtered_model.add(Conv2D(64, (3, 3), activation='relu'))
filtered_model.add(MaxPooling2D((2, 2)))

filtered_model.add(Conv2D(64, (3, 3), activation='relu'))
filtered_model.add(MaxPooling2D((2, 2)))

filtered_model.add(Conv2D(128, (3, 3), activation='relu'))
filtered_model.add(MaxPooling2D((2, 2)))

# Flatten the output and add dense layers
filtered_model.add(Flatten())
filtered_model.add(Dense(256, activation='relu'))
filtered_model.add(Dense(6, activation='softmax'))

# Compile the model
filtered_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Print the model summary
# filtered_model.summary()

# Train the model using the training data
filtered_model.fit(filtered_train_generator, epochs=epoch_number)

Found 7518 images belonging to 6 classes.
Found 74 images belonging to 6 classes.
Epoch 1/10
  3/235 [..............................] - ETA: 26:53 - loss: 1.7704 - accuracy: 0.1979

In [11]:
filtered_model.save('drive/MyDrive/Skin-detection/Original Images/Original Images/FOLDS/fold1/working/filtered_model.csv')
# Load the saved model
loaded_model = load_model('drive/MyDrive/Skin-detection/Original Images/Original Images/FOLDS/fold1/working/filtered_model.csv')

# Make predictions on the filtered test generator
predictions = filtered_model.predict(filtered_test_generator)

# Get the predicted class labels
predicted_labels = np.argmax(predictions, axis=1)
# predicted_labels

# print the disease predicted
for i in predicted_labels:
    # Iterate over the dictionary items and find the key for the desired value
    for key, value, in class_labels.items():
        if value == i:
            print("{} : {}".format(i, key))

# Loading an external data
image = Image.open("drive/MyDrive/Skin-detection/Original Images/Original Images/FOLDS/fold1/Test/Chickenpox/CHP_08_01.jpg")
# preprocess the input
image = image.resize((width_size, height_size))
image = np.array(image)
image = np.expand_dims(image, axis=0)
image_tensor = np.vstack([image])
predictions = loaded_model.predict(image_tensor)
# print a maxmum value
predicted_labels = np.argmax(predictions, axis=1)
# print the disease predicted
for i in predicted_labels:
    # Iterate over the dictionary items and find the key for the desired value
    for key, value in class_labels.items():
        if value == i:
            print("{} : {}".format(i, key))




3/3 [==============================] - 1s 137ms/step
3 : Healthy
5 : Monkeypox
2 : HFMD
2 : HFMD
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
1 : Cowpox
5 : Monkeypox
2 : HFMD
2 : HFMD
5 : Monkeypox
2 : HFMD
2 : HFMD
5 : Monkeypox
5 : Monkeypox
2 : HFMD
5 : Monkeypox
3 : Healthy
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
2 : HFMD
2 : HFMD
5 : Monkeypox
5 : Monkeypox
2 : HFMD
5 : Monkeypox
5 : Monkeypox
2 : HFMD
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
1 : Cowpox
5 : Monkeypox
5 : Monkeypox
2 : HFMD
2 : HFMD
2 : HFMD
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
2 : HFMD
2 : HFMD
3 : Healthy
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
2 : HFMD
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
2 : HFMD
5 : Monkeypox
5 : Monkeypox
2 : HFMD
5 : Monkeypox
3 : Healthy
5 : Monkeypox
5 : Monkeypox
5 : Monkeypox
1 : Cowpox
2 : HFMD
5 : Monkeypox
2 : HFMD
5 : Monkeypox
1/1 [============================

In [ ]:
# Show image
train = process_data(filtered_train_dir, TRAIN_FOLDER)
def show_images(data, isTest=False):
    f, ax = plt.subplots(5,5, figsize=(15,15))
    for i,data in enumerate(data[:25]):
        img_num = data[1]
        img_data = data[0]
        label = np.argmax(img_num)
        if label  == 1:
            str_label='Dog'
        elif label == 0:
            str_label='Cat'
        if(isTest):
            str_label="None"
        ax[i//5, i%5].imshow(img_data)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Label: {}".format(str_label))
    plt.show()

show_images(train)